In [1]:
import moldesign as mdt
from moldesign import units as u

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med

In [ ]:
mol = mdt.from_pdb('1yu8')


In [ ]:
mdmol = mdt.assign_forcefield(mol)

print "Done parameterizing"



Connecting to docker host at unix://var/run/docker.sock... done


Widget Javascript not detected.  It may not be installed or enabled properly.


In [ ]:
mdmol.set_energy_model(LAMMPSPotential)
print "Done setting energy model"


In [ ]:
hello = mdmol.calculate()
print "Done calculating"

In [ ]:
hello['potential_energy']

In [ ]:
mdmol.set_integrator(mdt.integrators.LAMMPSNvt,
                   timestep=1.0*u.fs,
                   temperature=300.0*u.kelvin,
                   frame_interval=5.0*u.fs)

In [ ]:
print mdmol.atoms[1004].position

In [ ]:
mdmol.run(20*u.fs)

In [ ]:
print mdmol.atoms[1004].position
print mdmol.atoms[1004].velocity